<a href="https://colab.research.google.com/github/Koushik-Pula/face-distance-detection/blob/main/face_distance_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi nest-asyncio uvicorn pyngrok opencv-python-headless


In [ ]:
!ngrok config add-authtoken 2rhj93L2xNq0JTijHyOSpbwHEGZ_5RujG7TdnQcH24qCaRivt

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from fastapi import FastAPI, WebSocket
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok
import base64
import cv2
import numpy as np
import logging
import os

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

FOCAL_LENGTH = 540
KNOWN_WIDTH = 0.15

def detect_face_and_distance(frame, focal_length):
    try:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        if len(faces) > 0:
            (x, y, w, h) = faces[0]
            distance = (KNOWN_WIDTH * focal_length) / w
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, f"{distance:.2f}m", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

            logger.info(f"Detected face at distance: {distance:.2f} meters")
            return distance, frame
        return None, frame
    except Exception as e:
        logger.error(f"Error in face detection: {str(e)}")
        return None, frame

@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()

    try:
        while True:
            data = await websocket.receive_json()

            if "image" not in data:
                await websocket.send_json({"error": "No image received"})
                continue

            try:
                img_data = base64.b64decode(data["image"])
                np_arr = np.frombuffer(img_data, np.uint8)
                frame = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

                if frame is None:
                    logger.error("Failed to decode image")
                    await websocket.send_json({"error": "Failed to decode image"})
                    continue

                distance, processed_frame = detect_face_and_distance(frame, FOCAL_LENGTH)

                _, buffer = cv2.imencode('.jpg', processed_frame)
                jpg_as_text = base64.b64encode(buffer).decode('utf-8')

                await websocket.send_json({
                    "image": jpg_as_text,
                    "distance": float(distance) if distance is not None else -1
                })

            except Exception as e:
                logger.error(f"Error processing frame: {str(e)}")
                await websocket.send_json({"error": f"Error processing frame: {str(e)}"})

    except Exception as e:
        logger.error(f"WebSocket error: {str(e)}")
    finally:
        logger.info("WebSocket connection closed")

if __name__ == "__main__":
    # Create a tunnel to the FastAPI backend
    public_url = ngrok.connect(8000)
    print(f"FastAPI app is running on: {public_url}")

    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)


FastAPI app is running on: NgrokTunnel: "https://9f27-34-134-167-235.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [379]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     ('14.139.177.158', 0) - "WebSocket /ws" [accepted]
INFO:     connection open
ERROR:__main__:WebSocket error: (<CloseCode.NO_STATUS_RCVD: 1005>, '')
INFO:     connection closed
INFO:     ('14.139.177.158', 0) - "WebSocket /ws" [accepted]
INFO:     connection open
ERROR:__main__:WebSocket error: (1001, '')
INFO:     connection closed
INFO:     ('14.139.177.158', 0) - "WebSocket /ws" [accepted]
INFO:     connection open
INFO:     ('14.139.177.158', 0) - "WebSocket /ws" [accepted]
INFO:     connection open
ERROR:__main__:WebSocket error: (1001, '')
INFO:     connection closed
INFO:     ('14.139.177.158', 0) - "WebSocket /ws" [accepted]
INFO:     connection open
ERROR:__main__:WebSocket error: (1001, '')
INFO:     connection closed
ERROR:__main__:WebSocket error: (<CloseCode.NO_STATUS_RCVD: